In [7]:
import os
import pandas as pd
from tqdm.notebook import tqdm

In [8]:
path = "/home/patel_zeel/compass24/exact_latlon"
files = os.listdir(path)
print(len(files))
df = pd.concat([pd.read_csv(os.path.join(path, f)) for f in os.listdir(path) if f.endswith('.csv')])
df.shape

10


(25565, 18)

In [9]:
df.head(2)

,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,top_left_x,top_left_y,bottom_right_x,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y
0,"30.79,75.87",0,zigzag,30.788065,75.871358,punjab,2.384894e+07,1.375857e+07,0.115034,0.045408,2.384887e+07,1.375854e+07,2.384900e+07,1.375859e+07,622.157760,744.50432,750.995840,795.36128
1,"30.79,75.87",1,zigzag,30.787651,75.875818,punjab,2.384935e+07,1.375861e+07,0.031660,0.100504,2.384934e+07,1.375856e+07,2.384937e+07,1.375867e+07,1084.514928,758.56144,1119.974352,871.12592


In [13]:
# csv to kml
import simplekml

fcbk = simplekml.Kml()
zigzag = simplekml.Kml()
i = 0
for lon, lat, class_name, state in tqdm(zip(df['lon'], df['lat'], df['class'], df['state'])):
    if class_name == 'fcbk':
        fcbk.newpoint(name=class_name, coords=[(lon, lat)], description=state)
    else:
        zigzag.newpoint(name=class_name, coords=[(lon, lat)], description=state)

fcbk.save("/home/patel_zeel/compass24/kml/fcbk.kml")
zigzag.save("/home/patel_zeel/compass24/kml/zigzag.kml")

0it [00:00, ?it/s]

In [14]:
df['class'].value_counts()

zigzag    19242
fcbk       6323
Name: class, dtype: int64

In [16]:
min_lat = 26.6202
max_lat = 27.1980
min_lon = 80.6611
max_lon = 81.2322

# filter data
lucknow_df = df[(df['lat'] >= min_lat) & (df['lat'] <= max_lat) & (df['lon'] >= min_lon) & (df['lon'] <= max_lon)]
lucknow_df.shape

(165, 18)

In [19]:
# csv to kml
lucknow_kml = simplekml.Kml()

for lon, lat, class_name, state in tqdm(zip(lucknow_df['lon'], lucknow_df['lat'], lucknow_df['class'], lucknow_df['state'])):
    if class_name == 'fcbk':
        lucknow_kml.newpoint(name="", coords=[(lon, lat)], description=f"{state} - {class_name}")
    else:
        lucknow_kml.newpoint(name="", coords=[(lon, lat)], description=f"{state} - {class_name}")
        
lucknow_kml.save("/home/patel_zeel/compass24/kml/lucknow.kml")

0it [00:00, ?it/s]

In [18]:
lucknow_df['class'].value_counts()

zigzag    97
fcbk      68
Name: class, dtype: int64

In [24]:
from fastkml import kml

def open_kml(file_path):
    with open(file_path, 'rt', encoding="utf-8") as file:
        doc = file.read()
    doc = doc.split('?>')[-1]  # Remove encoding declaration
    k = kml.KML()
    k.from_string(doc)
    return k

kml = open_kml("/home/patel_zeel/compass24/kml/grids_lucknow-1.kml")

In [34]:
len(list(list(kml.features())[0].features()))[:1800]

3600